In [174]:
import os
import glob
import parsedatetime
import pandas as pd
from datetime import datetime
from statistics import mode
wd = "/Volumes/som/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Viral Shah/JDRF TIR/3. Data Collection/Cleaned Final Data/"
cal = parsedatetime.Calendar()

In [ ]:
# Dict for storing data
df = {"id":[],"timepoint":[],"tir":[]}
# Calculate CGM values, etc. for each person
folders = os.listdir(wd + "Cases_T1D+DR")
folders = [f for f in folders if "DS_Store" not in f]
for fol in folders:
    # Get ID
    subject_id = [int(i) for i in fol.split() if i.isdigit()][0]
    # Calculate CGM metrics for each CSV file
    files = os.listdir(wd + "Cases_T1D+DR/" + fol)
    csvs = [f for f in files if ".csv" in f]
    for c in csvs:
        vis = c.split("_")[0]
        vis = [int(i) for i in vis.split() if i.isdigit()][0]
        cgm = pd.read_csv(wd + "Cases_T1D+DR/" + fol + "/" + c)
        # Get timestamp and glucose columns, format
        if "Timestamp (YYYY-MM-DDThh:mm:ss)" in cgm.columns:
            cgm = cgm[["Timestamp (YYYY-MM-DDThh:mm:ss)","Glucose Value (mg/dL)"]]
        elif cgm.shape[1] > 40:
            start = cgm.loc[cgm['Unnamed: 2'] == "Sensor"].index[0]
            cgm.columns = cgm.iloc[start+1,:]
            cgm = cgm.iloc[start+2:,:]
            cgm.reset_index(inplace = True)
            cgm["timestamp"] = cgm["Date"] + " " + cgm["Time"]
            cgm = cgm[["timestamp","Sensor Glucose (mg/dL)"]]
        cgm.columns = ["timestamp","glucose"]
        cgm.dropna(inplace = True)
        # Glucose to numeric replace "high" and "low" with limits of detection
        cgm["glucose"].replace("High",400,inplace = True)
        cgm["glucose"].replace("Low",40,inplace = True)
        cgm["glucose"] = [float(g) for g in cgm["glucose"]]
        cgm["timestamp"] = [t.replace("T", " ") for t in cgm["timestamp"]]
        cgm["timestamp"] = [cal.parse(t)[0] for t in cgm["timestamp"]]
        cgm["timestamp"] = [datetime(*t[:6]) for t in cgm["timestamp"]]
        # Find sampling interval
        s = abs(mode(cgm["timestamp"].diff()).total_seconds())
        # Re-index and expand to every 5 minutes
        cgm.set_index("timestamp",inplace = True)
        cgm = cgm[~cgm.index.duplicated(keep='first')]
        cgm = cgm.resample(str(s / 60) + "T").ffill()
        total_r = cgm["glucose"].notna().sum()
        # TIR
        tir = [g for g in cgm["glucose"] if g >= 70 and g <= 180]
        # Append to results dict
        df["id"].append(subject_id)
        df["timepoint"].append(vis)
        df["tir"].append(round(len(tir) / total_r * 100,2))

In [ ]:
wd + "Cases_T1D+DR/" + fol + "/" + c